In [66]:
%%writefile dlnd_p2.yml

---
settings:

  # Where to get the data
  cifar: &cifar
    url: "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    checksum: "6d958be074577803d12ecdefd02955f39262c83c16fe9348329d7fe0b5c001ce"
    path: "~/kur"           # if kur does not have normalization or one-hot-encoding, ##################
                            # without normalization and one-hot-encoding, the performance will be hurt, right? ####
  # Backend to use                                      
  backend:
    name: keras

  # Hyperparameters
  cnn:
    kernels: 20
    size: [5, 5]
    strides: [2, 2]                

  pool:
    size: [2,2]
    strides: [2,2]
    type: "max"                         # must use a string here, {max} won't work, doc didn't say it #############

model:
  - input: images                       # images are normalized from 0 to 1, labels are one-hot-encoding ##########
  - convolution:                        # does kur do normalize and one-hot-encoding under the hood? ##############
      kernels: "{{cnn.kernels}}"
      size: "{{cnn.size}}"
      strides: "{{ cnn.strides }}"
  - activation: relu
  - pool:
      size: "{{pool.size}}"
      strides: "{{pool.strides}}"
      type: "{{pool.type}}"             
  - flatten:
  - dense: 15                           # p2 want a dropout applied here, but kur does not have yet?? ############
  - dense: 10
  - activation: softmax
    name: labels

train:
  data:
    - cifar:
        <<: *cifar
        parts: [1]                      # only use dataset part 1 to train
  provider:
    batch_size: 128
    num_batches: 1000                   # the entire part 1 will be used
  log: t2/cifar-log                   
  epochs: 20
  weights:
    initial: t2/cifar.best.valid.w    
    best: t2/cifar.best.train.w
    last: t2/cifar.last.w

  optimizer:
    name: adam
    learning_rate: 0.001

validate:
  data:
    - cifar:
       <<: *cifar
       parts: 5                          # only use dataset part 5 as validation set
  provider:
    num_batches: 50                      # the project 2 only used 5000 data points as validation set
  weights: t2/cifar.best.valid.w

test: &test
  data:
    - cifar:
       <<: *cifar
       parts: test
  weights: t2/cifar.best.valid.w
  provider:
    num_batches: 1000                     # the entire part test will be used

evaluate:
  <<: *test
  destination: t2/cifar.results.pkl

loss:
  - target: labels                        # in the project: training loss and valid_accuracy are printed #############
    name: categorical_crossentropy        # this should be a matter of personal taste, won't really affect anything##
...


Overwriting dlnd_p2.yml


In [67]:
%pwd

'/Users/Natsume/Downloads/kur_road'

In [68]:
%ls

Kur_Road.ipynb     dlnd_p2.yml        t1/
cifar-log/         dlnd_p2_kur.ipynb  t2/


In [69]:
!kur -v train dlnd_p2.yml

[INFO 2017-02-28 21:20:36,347 kur.kurfile:638] Parsing source: dlnd_p2.yml, included by top-level.
[INFO 2017-02-28 21:20:36,361 kur.kurfile:79] Parsing Kurfile...
[INFO 2017-02-28 21:20:36,393 kur.loggers.binary_logger:63] Loading log data: t2/cifar-log
[INFO 2017-02-28 21:20:43,248 kur.providers.batch_provider:54] Batch size set to: 128
[INFO 2017-02-28 21:20:43,248 kur.providers.batch_provider:60] Maximum number of batches set to: 1000
[INFO 2017-02-28 21:20:49,334 kur.providers.batch_provider:54] Batch size set to: 32
[INFO 2017-02-28 21:20:49,335 kur.providers.batch_provider:60] Maximum number of batches set to: 50
[INFO 2017-02-28 21:20:49,335 kur.backend.backend:80] Creating backend: keras
[INFO 2017-02-28 21:20:49,335 kur.backend.backend:83] Backend variants: none
[INFO 2017-02-28 21:20:49,335 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-02-28 21:20:50,349 kur.backend.keras_backend:191] Keras is loaded. The backend is: theano
[IN